# Análise do Catálogo da Netflix


**SME0221 Inferência Estatística - Cibele Russo - ICMC/USP - São Carlos SP**

Alexandre Souza - 12559506

Eduardo Zaffari Monteiro - 12559490

Lucas Ivars Cadima Ciziks - 12559472

## Introdução
Em 2021, a indústria de entretenimento audiovisual provou que conseguiu superar a pandemia do COVID-19 no que tange ao seu faturamento total. Segundo o relatório anual da Motion Picture Association (MPA), no ano passado foram arrecadados US$ 99,7 bilhões, número que supera os US$ 98,1 bilhões de 2019, ano anterior à crise sanitária. 

Da arrecadação total, 72% refere-se ao consumo de mídia digital, o equivalente a US$ 71,9 bilhões, impulsionado pelos serviços de mídia digital (serviços de *streaming* e *TV Everywhere*), que contavam com mais de 1,3 bilhão de assinaturas ativas no último ano, das quais 221,6 milhões eram da Netflix.

Em se tratando da Netflix, que hoje possui 45,2% da fatia de mercado de serviços de *streaming*, aproximadamente 17,8% de suas assinaturas são da América Latina, contra os 33,2% da EMEA, região que corresponde a Europa, Oriente Médio e África. Contudo, ainda segundo o relatório da MPA, o mercado de serviços digitais aumentou 29% nos países latino-americanos em 2021 (quando comparado a 2020), em contraste com os  21% da região comercial EMEA.

Tendo isso em vista, o objetivo deste relatório é tentar provar, por meio da análise exploratória dos dados e teste de hipóteses, que houve maior investimento em produções de origem europeia em detrimento das de origem latino-americana, explicitando a desigualdade cultural entre as duas regiões.

## Desenvolvimento

In [1]:
# Importação de Bibliotecas
import json
import pandas as pd
import numpy as np
from scipy.stats import t, shapiro, boxcox
import plotly.express as px

### Coleta e Limpeza dos dados

In [2]:
# Carregando Dados referentes ao Catálogo da Netflix
netflix_raw = pd.read_csv('https://raw.githubusercontent.com/Cizika/netflix-analysis/main/netflix_titles.csv', index_col=0)
netflix_raw.head()

,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
show_id,,,,,,,,,,,
s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [3]:
# Lista de Países Europeus
european_list = ["Albania","Andorra","Åland","Austria","Belarus","Belgium",
                 "Bosnia and Herzegovina","Bulgaria","Croatia","Cyprus",
                 "Czech Republic","Denmark","Estonia",
                 "Finland","France","Germany","Gibraltar","Greece",
                 "Hungary","Iceland","Ireland", "Italy","Jersey","Latvia",
                 "Liechtenstein","Lithuania","Luxembourg","Macedonia","Malta",
                 "Moldova","Monaco","Montenegro","Netherlands","Norway","Poland",
                 "Portugal","Romania", "Russia","Serbia","Spain","Sweden",
                 "Switzerland","Slovakia","Slovenia","Ukraine","United Kingdom"]

In [4]:
# Lista de Países Latinos
latin_list = ["Costa Rica","Cuba","Haiti","Panama","Saint Bartélemy","El Salvador",
              "Saint Martin","Argentina","Belize","Bolivia","Brazil","Chile",
              "Colombia","Ecuador","Dominican Republic","Guatemala","Honduras",
              "Mexico","Nicaragua","Paraguay","Peru","Puerto Rico","Uruguay",
              "Venezuela"]

In [5]:
# Excluíndo valores nulos e vazios dos dados
netflix_df = netflix_raw[["country","date_added"]].dropna()

# Tratando a data de adição do filme ao catálogo somento pelo seu ano
netflix_df["date_added"] = netflix_df["date_added"].apply(lambda x: int(x.split(", ")[1]))

# Exibindo dados originais
netflix_df

,country,date_added
show_id,,
s1,United States,2021
s2,South Africa,2021
s5,India,2021
s8,"United States, Ghana, Burkina Faso, United Kin...",2021
s9,United Kingdom,2021
...,...,...
s8802,"United Arab Emirates, Jordan",2016
s8803,United States,2019
s8805,United States,2019


### Coletando Amostra

In [6]:
# Coletando amostra dos filmes adicionados em 2021
amostra_raw = netflix_df[netflix_df["date_added"] == 2021].copy()

# Individualizando país por linha (Tratando filmes com mais de um país de origem)
amostra_raw = amostra_raw.assign(country=amostra_raw["country"].str.split(", ")).explode("country")

# Exibindo amostra raw
amostra_raw

,country,date_added
show_id,,
s1,United States,2021
s2,South Africa,2021
s5,India,2021
s8,United States,2021
s8,Ghana,2021
...,...,...
s8321,Sweden,2021
s8321,Norway,2021
s8435,United States,2021


In [7]:
# A amostra será o Número de Filmes por País
amostra = amostra_raw.groupby("country").size().to_frame()

# Excluíndo valores vazios da amostra
amostra.drop("", inplace=True)

# Renomeando Colunas da Amostra
amostra.reset_index(inplace=True)
amostra.rename(columns={"country":"País", 0: "Número de Filmes" }, inplace=True)

# Exibindo amostra
amostra

,País,Número de Filmes
0,Algeria,1
1,Argentina,12
2,Australia,24
3,Austria,1
4,Bangladesh,1
...,...,...
67,"United Kingdom,",1
68,United States,627
69,Uruguay,2
70,Venezuela,1


In [8]:
# Dicotomizando Amostra por Variável Qualitativa Binária "Região"
# As Regiões de análise serão somente América Latina e Europa

def somente_europa_latinos(row):
  if row["País"] in latin_list:
    return "América Latina"
  elif row["País"] in european_list:
    return "Europa"

# Filtrando apenas por países Europeus e Latinos
amostra["Região"] = amostra.apply(somente_europa_latinos, axis=1)
amostra.dropna(inplace=True)

# Exibindo Amostra com Variável Quantitativa (Número de Filmes)
# E Variável Qualitativa Binária (Região)
amostra.set_index(["Região", "País"]).sort_index(level=0)

Número de Filmes
Região         País                            
América Latina Argentina                     12
               Brazil                        11
               Chile                          2
               Colombia                       9
               Mexico                        21
               Uruguay                        2
               Venezuela                      1
Europa         Austria                        1
               Belgium                        9
               Bulgaria                       2
               Czech Republic                 4
               Denmark                        5
               Finland                        2
               France                        60
               Germany                       40
               Greece                         1
               Hungary                        3
               Iceland                        1
               Ireland                        5
               Italy                         16
               Luxembourg                     1
               Netherlands                    5
               Norway                         3
               Poland                         6
               Portugal                       2
               Romania                        5
               Russia                         4
               Spain                         33
               Sweden                         4
               Switzerland                    3
               United Kingdom               119

### Análise Exploratória dos Dados

In [9]:
# Análise Descritiva dos Latinos
amostra_latinos = amostra[amostra["Região"] == "América Latina"]

amostra_latinos.describe()

,Número de Filmes
count,7.000000
mean,8.285714
std,7.250616
min,1.000000
25%,2.000000
50%,9.000000
75%,11.500000
max,21.000000


In [10]:
# Análise Descritiva dos Europeus
amostra_europeus = amostra[amostra["Região"] == "Europa"]

amostra_europeus.describe()

,Número de Filmes
count,24.000000
mean,13.916667
std,26.675044
min,1.000000
25%,2.000000
50%,4.000000
75%,6.750000
max,119.000000


In [11]:
# Esboçando Boxplot com dados da Amostra dicotomizada
boxplot = px.box(amostra, x='Região', y= 'Número de Filmes', color="Região")
boxplot.show()

In [12]:
# Plotando Histograma com dados da Amostra dicotomizada
histograma = px.histogram(amostra, x='Número de Filmes', nbins=50, color="Região")
histograma.show()

### Teste de Normalidade

#### Teste de Shapiro-Wilk

In [13]:
# Função para verificar normalidade de uma amostra
def verificar_normalidade(amostra):
  shapiro_stats, p_valor = shapiro(amostra)

  print(f"Valor p encontrado: {p_valor}")

  if p_valor < 0.05:
    print("Portanto, há evidências para recusar a normalidade, segundo o Teste de Shapiro-Wilk")
  else:
    print("Portanto, não há evidências para recusar a normalidade, segundo o Teste de Shapiro-Wilk")

In [14]:
# Verificando normalidade da amostra original
verificar_normalidade(amostra["Número de Filmes"])

Valor p encontrado: 5.8314886253185705e-09
Portanto, há evidências para recusar a normalidade, segundo o Teste de Shapiro-Wilk


#### Normalizando os dados através de Box-Cox

In [15]:
# Aplicando boxcox sobre a amostra
amostra_normalizada, lam = boxcox(amostra["Número de Filmes"])

# Verificando normalidade da nova amostra
verificar_normalidade(amostra_normalizada)

Valor p encontrado: 0.2368444800376892
Portanto, não há evidências para recusar a normalidade, segundo o Teste de Shapiro-Wilk


In [16]:
# Armazenando valores normalizados na amostra
amostra["Número de Filmes Normalizado"] = amostra_normalizada

# Exibindo Amostra preparada para o Teste de Hipóteses
amostra

,País,Número de Filmes,Região,Número de Filmes Normalizado
1,Argentina,12,América Latina,1.834409
3,Austria,1,Europa,0.000000
5,Belgium,9,Europa,1.677128
6,Brazil,11,América Latina,1.788064
7,Bulgaria,2,Europa,0.634699
13,Chile,2,América Latina,0.634699
15,Colombia,9,América Latina,1.677128
16,Czech Republic,4,Europa,1.165454
17,Denmark,5,Europa,1.317088
20,Finland,2,Europa,0.634699


### Teste para Comparação de Médias entre Duas Populações Independentes

Definindo as seguintes Variáveis Aleatórias para realizar os Teste:

$L: $ Número de filmes produzidos por países latinos no catálogo da Netflix;

$E: $ Número de filmes produzidos por países europeus no catálogo da Netflix.

Considerando a **média populacional** de filmes produzidos por país latino como $\mu_L$ e a **média populacional** de filmes produzidos por país europeu como $\mu_E$. Para a comparação das médias, define-se a estatística $\overline{D}$ tal que:

$$\overline{D} = \overline{L} - \overline{E} \sim N\left(\mu_D = \mu_L - \mu_E, \sigma^2_D = \frac{\sigma^2_L}{n_L} + \frac{\sigma^2_E}{n_E} \right)$$

Assim, considera-se o seguinte teste de hipóteses:

*   $H_0: \mu_D = 0$ contra
*   $H_1: \mu_D \neq 0$

In [17]:
# Calculando o tamanho da amostra da América Latina
n_latinos = amostra_latinos["Número de Filmes"].count()

# Calculando o tamanho da amostra da Europa
n_europeus = amostra_europeus["Número de Filmes"].count()

In [18]:
# Calculando a média de filmes por País da América Latina
mean_latinos = amostra_latinos["Número de Filmes"].mean()

print(f"Em 2021, cada país latino produziu, em média, {mean_latinos} filmes.")

# Calculando a média de filmes por País da Europa
mean_europa = amostra_europeus["Número de Filmes"].mean()

print(f"Em 2021, cada país europeu produziu, em média, {mean_europa} filmes.")

Em 2021, cada país latino produziu, em média, 8.285714285714286 filmes.
Em 2021, cada país europeu produziu, em média, 13.916666666666666 filmes.


In [19]:
# Calculando Variância Amostral dos Latinos
var_latinos = amostra_latinos["Número de Filmes"].var()

print(f"Em 2021, a variância amostral encontrada para os países latinos foi {var_latinos}.")

# Calculando Variância Amostral dos Europeus
var_europeus = amostra_europeus["Número de Filmes"].var()

print(f"Em 2021, a variância amostral encontrada para os países europeus foi {var_europeus}.")

Em 2021, a variância amostral encontrada para os países latinos foi 52.57142857142858.
Em 2021, a variância amostral encontrada para os países europeus foi 711.5579710144928.


In [20]:
# Calculando o nu de graus de liberdade
var_latinos_divisao = var_latinos/ n_latinos
var_europeus_divisao = var_europeus/ n_europeus

nu = (var_latinos_divisao + var_europeus_divisao) / (var_latinos_divisao**2 / (n_latinos - 1) + (var_europeus_divisao**2 / (n_europeus - 1)))

In [21]:
# Sob H0
mu_D = 0

tc_1 = t.ppf(0.025, nu)
tc_2 = t.ppf(0.975, nu)
                             
# print('RC = {t >', np.round(tc,2), '}')
tc_1

-25.236437997371258

In [22]:
tc_2

25.23643799737123